# L4: Tools for a Customer Outreach Campaign

In this lesson, you will learn more about Tools. You'll focus on three key elements of Tools:
- Versatility
- Fault Tolerance
- Caching

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM
- [Serper](https://serper.dev)

In [2]:
from crewai import Agent, Task, Crew

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [26]:
import os
from utils import get_openai_api_key, pretty_print_result
from utils import get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4o-mini'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## Creating Agents

In [27]:
sales_rep_agent = Agent(
    role="Sales Representative",
    goal="Identify high-value leads that match "
         "our ideal customer profile",
    backstory=(
        "As a part of the dynamic sales team at CrewAI, "
        "your mission is to scour "
        "the digital landscape for potential leads. "
        "Armed with cutting-edge tools "
        "and a strategic mindset, you analyze data, "
        "trends, and interactions to "
        "unearth opportunities that others might overlook. "
        "Your work is crucial in paving the way "
        "for meaningful engagements and driving the company's growth."
    ),
    allow_delegation=False,
    verbose=True
)

In [28]:
lead_sales_rep_agent = Agent(
    role="Lead Sales Representative",
    goal="Nurture leads with personalized, compelling communications",
    backstory=(
        "Within the vibrant ecosystem of CrewAI's sales department, "
        "you stand out as the bridge between potential clients "
        "and the solutions they need."
        "By creating engaging, personalized messages, "
        "you not only inform leads about our offerings "
        "but also make them feel seen and heard."
        "Your role is pivotal in converting interest "
        "into action, guiding leads through the journey "
        "from curiosity to commitment."
    ),
    allow_delegation=False,
    verbose=True
)

## Creating Tools

### crewAI Tools

In [29]:
from crewai_tools import DirectoryReadTool, \
                         FileReadTool, \
                         SerperDevTool

In [30]:
directory_read_tool = DirectoryReadTool(directory='./instructions')
file_read_tool = FileReadTool()
search_tool = SerperDevTool()

### Custom Tool
- Create a custom tool using crewAi's [BaseTool](https://docs.crewai.com/core-concepts/Tools/#subclassing-basetool) class

In [31]:
from crewai.tools import BaseTool

- Every Tool needs to have a `name` and a `description`.
- For simplicity and classroom purposes, `SentimentAnalysisTool` will return `positive` for every text.
- When running locally, you can customize the code with your logic in the `_run` function.

In [32]:
class SentimentAnalysisTool(BaseTool):
    name: str ="Sentiment Analysis Tool"
    description: str = ("Analyzes the sentiment of text "
         "to ensure positive and engaging communication.")
    
    def _run(self, text: str) -> str:
        # Your custom code tool goes here
        return "positive"

In [33]:
sentiment_analysis_tool = SentimentAnalysisTool()

## Creating Tasks

- The Lead Profiling Task is using crewAI Tools.

In [34]:
lead_profiling_task = Task(
    description=(
        "Conduct an in-depth analysis of {lead_name}, "
        "a company in the {industry} sector "
        "that recently showed interest in our solutions. "
        "Utilize all available data sources "
        "to compile a detailed profile, "
        "focusing on key decision-makers, recent business "
        "developments, and potential needs "
        "that align with our offerings. "
        "This task is crucial for tailoring "
        "our engagement strategy effectively.\n"
        "Don't make assumptions and "
        "only use information you absolutely sure about."
    ),
    expected_output=(
        "A comprehensive report on {lead_name}, "
        "including company background, "
        "key personnel, recent milestones, and identified needs. "
        "Highlight potential areas where "
        "our solutions can provide value, "
        "and suggest personalized engagement strategies."
    ),
    tools=[directory_read_tool, file_read_tool, search_tool],
    agent=sales_rep_agent,
)

- The Personalized Outreach Task is using your custom Tool `SentimentAnalysisTool`, as well as crewAI's `SerperDevTool` (search_tool).

In [35]:
personalized_outreach_task = Task(
    description=(
        "Using the insights gathered from "
        "the lead profiling report on {lead_name}, "
        "craft a personalized outreach campaign "
        "aimed at {key_decision_maker}, "
        "the {position} of {lead_name}. "
        "The campaign should address their recent {milestone} "
        "and how our solutions can support their goals. "
        "Your communication must resonate "
        "with {lead_name}'s company culture and values, "
        "demonstrating a deep understanding of "
        "their business and needs.\n"
        "Don't make assumptions and only "
        "use information you absolutely sure about."
    ),
    expected_output=(
        "A series of personalized email drafts "
        "tailored to {lead_name}, "
        "specifically targeting {key_decision_maker}."
        "Each draft should include "
        "a compelling narrative that connects our solutions "
        "with their recent achievements and future goals. "
        "Ensure the tone is engaging, professional, "
        "and aligned with {lead_name}'s corporate identity."
    ),
    tools=[sentiment_analysis_tool, search_tool],
    agent=lead_sales_rep_agent,
)

## Creating the Crew

In [36]:
crew = Crew(
    agents=[sales_rep_agent, 
            lead_sales_rep_agent],
    
    tasks=[lead_profiling_task, 
           personalized_outreach_task],
	
    verbose=True,
	memory=True
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [ ]:
inputs = {
    "lead_name": "DeepLearningAI",
    "industry": "Online Learning Platform",
    "key_decision_maker": "Andrew Ng",
    "position": "CEO",
    "milestone": "product launch"
}

result = crew.kickoff(inputs=inputs)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 5e19fef2-efb3-4edd-ace4-c0b213b9b957                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭────────────────────────────────────────────── 🧠 Retrieved Memory ──────────────────────────────────────────────╮
│                                                                                                                 │
│  Recent Insights:                                                                                               │
│  - Thought: The search results have provided detailed information about key personnel at DeepLearningAI, such   │
│  as Andrew Ng, Shubh Bariya, and Nicole Angco, among others. With this information, we can identify potential   │
│  decision-makers relevant to our engagement strategy.                                                           │
│                                                                                                                 │
│  Final Answer: DeepLearningAI is an education technology company founded in 2017 by machine learning and        │
│  education pioneer Andrew Ng. The company is committed to making world-class AI education accessible glob...    │
│                                                                                                                 │
╰─────────────────────────────────────────── Retrieval Time: 1140.77ms ───────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Sales Representative                                                                                    │
│                                                                                                                 │
│  Task: Conduct an in-depth analysis of DeepLearningAI, a company in the Online Learning Platform sector that    │
│  recently showed interest in our solutions. Utilize all available data sources to compile a detailed profile,   │
│  focusing on key decision-makers, recent business developments, and potential needs that align with our         │
│  offerings. This task is crucial for tailoring our engagement strategy effectively.                             │
│  Don't make assumptions and only use information you absolutely sure about.                                     │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

...



╭────────────────────────────── Execution Traces ──────────────────────────────╮
│                                                                              │
│  🔍 Detailed execution traces are available!                                 │
│                                                                              │
│  View insights including:                                                    │
│    • Agent decision-making process                                           │
│    • Task execution flow and timing                                          │
│    • Tool usage details                                                      │
│                                                                              │
╰──────────────────────────────────────────────────────────────────────────────╯


Would you like to view your execution traces? [y/N] (20s timeout): 



╭────────────────────────── Tracing Preference Saved ──────────────────────────╮
│                                                                              │
│  Info: Tracing has been disabled.                                            │
│                                                                              │
│  Your preference has been saved. Future Crew/Flow executions will not        │
│  collect traces.                                                             │
│                                                                              │
│  To enable tracing later, do any one of these:                               │
│  • Set tracing=True in your Crew/Flow code                                   │
│  • Set CREWAI_TRACING_ENABLED=true in your project's .env file               │
│  • Run: crewai traces enable                                                 │
│                                                                              │
╰─────────────────────────

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 5e19fef2-efb3-4edd-ace4-c0b213b9b957                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: Subject: Empowering DeepLearningAI’s Next-Gen AI Education with Advanced Integration Solutions   │
│                                                                                                                 │
│  Hi Andrew,                                                                                                     │
│                                                                                                                 │
│  I hope this message finds you well. I've been closely following DeepLearningAI’s impressive strides in AI      │
│  education, especially the recent launch of the "ChatGPT Prompt Engineering for Developers" course. It's        │
│  exciting to see DeepLearningAI lead with such innovative content, truly embodying your vision of making        │
│  high-level AI education widely accessible.                                                                     │
│                                                                                                                 │
│  At CrewAI, we specialize in enhancing educational platforms through AI-driven analytics and personalized       │
│  learning experiences. Considering the direction DeepLearningAI is headed, I believe our solutions could be     │
│  incredibly beneficial. Our advanced analytics can seamlessly integrate with your current offerings, providing  │
│  deeper insights into course engagement and effectiveness, and our personalization algorithms are designed to   │
│  adapt to individual learning paces and styles, potentially increasing student retention and satisfaction.      │
│                                                                                                                 │
│  Moreover, given your dedication to staying ahead in AI and machine learning arenas, I see a strategic          │
│  opportunity for us to collaborate on research and development. This could not only amplify DeepLearningAI’s    │
│  educational impact but also leverage our cutting-edge technological resources to push the boundaries of        │
│  what's possible in AI education.                                                                               │
│                                                                                                                 │
│  Would you be open to discussing this further? I would be delighted to explore potential synergies in a short   │
│  call at your convenience. Looking forward to the possibility of collaborating towards a transformative         │
│  educational experience.                                                                                        │
│                                                                                                                 │
│  Warm regards,                                                                                                  │
│                                                                                                                 │
│  [Your Name]                                                                                                    │
│  Lead Sales Representative, CrewAI                                                                              │
│                                                       

╭──────────────────────────────────────────────── Tracing Status ─────────────────────────────────────────────────╮
│                                                                                                                 │
│  Info: Tracing is disabled.                                                                                     │
│                                                                                                                 │
│  To enable tracing, do any one of these:                                                                        │
│  • Set tracing=True in your Crew/Flow code                                                                      │
│  • Set CREWAI_TRACING_ENABLED=true in your project's .env file                                                  │
│  • Run: crewai traces enable                                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

- Display the final result as Markdown.

In [38]:
from IPython.display import Markdown
Markdown(result.raw)

Subject: Empowering DeepLearningAI’s Next-Gen AI Education with Advanced Integration Solutions

Hi Andrew,

I hope this message finds you well. I've been closely following DeepLearningAI’s impressive strides in AI education, especially the recent launch of the "ChatGPT Prompt Engineering for Developers" course. It's exciting to see DeepLearningAI lead with such innovative content, truly embodying your vision of making high-level AI education widely accessible.

At CrewAI, we specialize in enhancing educational platforms through AI-driven analytics and personalized learning experiences. Considering the direction DeepLearningAI is headed, I believe our solutions could be incredibly beneficial. Our advanced analytics can seamlessly integrate with your current offerings, providing deeper insights into course engagement and effectiveness, and our personalization algorithms are designed to adapt to individual learning paces and styles, potentially increasing student retention and satisfaction.

Moreover, given your dedication to staying ahead in AI and machine learning arenas, I see a strategic opportunity for us to collaborate on research and development. This could not only amplify DeepLearningAI’s educational impact but also leverage our cutting-edge technological resources to push the boundaries of what's possible in AI education.

Would you be open to discussing this further? I would be delighted to explore potential synergies in a short call at your convenience. Looking forward to the possibility of collaborating towards a transformative educational experience.

Warm regards,

[Your Name]
Lead Sales Representative, CrewAI